# Simulador Balístico com PINN
Este notebook utiliza redes neurais PINN (Physics-Informed Neural Networks) com as equações de McCoy para prever a trajetória e o alcance máximo de um projétil, utilizando dados reais de arrasto fornecidos por radar (Radar Weibel).

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Definição do Modelo PINN

In [ ]:
class PINN(nn.Module):
    def __init__(self):
        super(PINN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(2, 64),
            nn.Tanh(),
            nn.Linear(64, 64),
            nn.Tanh(),
            nn.Linear(64, 2)  # Saída: [x(t), y(t)]
        )

    def forward(self, t, v0):
        input_tensor = torch.cat((t, v0), dim=1)
        return self.net(input_tensor)

## Função de Perda com as Equações de McCoy

In [ ]:
def loss_function(model, t, v0, m, A, rho, Cd_func, g=9.81):
    t.requires_grad = True
    v0.requires_grad = True
    out = model(t, v0)
    x = out[:, 0:1]
    y = out[:, 1:2]

    dx_dt = torch.autograd.grad(x, t, grad_outputs=torch.ones_like(x), create_graph=True)[0]
    dy_dt = torch.autograd.grad(y, t, grad_outputs=torch.ones_like(y), create_graph=True)[0]

    v = torch.sqrt(dx_dt**2 + dy_dt**2)
    Cd = Cd_func(v.detach().numpy())
    Cd = torch.tensor(Cd, dtype=torch.float32).reshape(-1, 1)

    Fd = 0.5 * Cd * rho * A * v ** 2
    ax = -Fd * (dx_dt / v) / m
    ay = -Fd * (dy_dt / v) / m - g

    ddx_dt = torch.autograd.grad(dx_dt, t, grad_outputs=torch.ones_like(dx_dt), create_graph=True)[0]
    ddy_dt = torch.autograd.grad(dy_dt, t, grad_outputs=torch.ones_like(dy_dt), create_graph=True)[0]

    loss = torch.mean((ddx_dt - ax)**2 + (ddy_dt - ay)**2)
    return loss

## Leitura dos Dados do Radar Weibel

In [ ]:
weibel_data = pd.read_csv("dados_weibel.txt", sep=r"\s+", header=None, names=["serie", "tempo", "mach", "Cd"])
weibel_data["Cd_suave"] = weibel_data["Cd"].rolling(window=9, center=True, min_periods=1).mean()

def interpolar_cd_tensor(mach):
    return np.interp(mach, weibel_data["mach"], weibel_data["Cd_suave"])

## Definições e Treinamento da Rede PINN

In [ ]:
m_proj = 0.04065
d_proj = 0.01298
A_proj = np.pi * (d_proj / 2) ** 2
rho = 1.225
v0_val = 905.0
v0_tensor = torch.full((100, 1), v0_val)
t_tensor = torch.linspace(0, 5, 100).reshape(-1, 1)

model = PINN()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
losses = []

for epoch in range(5000):
    optimizer.zero_grad()
    loss = loss_function(model, t_tensor, v0_tensor, m_proj, A_proj, rho, interpolar_cd_tensor)
    loss.backward()
    optimizer.step()
    losses.append(loss.item())
    if epoch % 500 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

## Resultados: Gráficos de Perda e Trajetória

In [ ]:
plt.figure()
plt.plot(losses)
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Evolução da Perda (PINN)")
plt.grid(True)
plt.show()

out_final = model(t_tensor, v0_tensor).detach().numpy()
x_final = out_final[:, 0]
y_final = out_final[:, 1]
alcance_max = np.max(x_final)
print(f"Alcance máximo previsto pela PINN: {alcance_max:.2f} m")

plt.figure()
plt.plot(x_final, y_final)
plt.xlabel("x (m)")
plt.ylabel("y (m)")
plt.title("Trajetória prevista pela PINN")
plt.grid(True)
plt.show()